In [1]:
#data ingestion
from langchain_community.document_loaders import TextLoader, WebBaseLoader
import bs4

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
loader = TextLoader("schema_info.txt")
text_document = loader.load()

In [3]:
text_document

[Document(metadata={'source': 'schema_info.txt'}, page_content='Table: brand_sales\n  - brandName (varchar(255))\n  - NoOfOrders (int(11))\n  - Sales (decimal(10,2))\n  - AOV (decimal(10,2))\n  - orderDate (date)\n\nTable: category_data\n  - subCategoryOf (varchar(255))\n  - Sales (decimal(10,2))\n  - orderDate (date)\n\nTable: category_sales\n  - subCategoryOf (varchar(255))\n  - Sales (decimal(10,2))\n  - orderDate (date)\n\nTable: product_sales\n  - productName (varchar(255))\n  - NoOfOrders (int(11))\n  - Sales (decimal(10,2))\n  - QuantitySold (int(11))\n  - orderDate (date)\n\nTable: sales_data\n  - invoice (varchar(255))\n  - storeInvoice (varchar(255))\n  - orderDate (date)\n  - time (time)\n  - productId (int(11))\n  - productName (varchar(255))\n  - barcode (varchar(255))\n  - quantity (int(11))\n  - sellingPrice (decimal(10,2))\n  - discountAmount (decimal(10,2))\n  - totalProductPrice (decimal(10,2))\n  - deliveryFee (decimal(10,2))\n  - HSNCode (varchar(255))\n  - GST (dec

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [9]:
# web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()


ConnectionError: HTTPSConnectionPool(host='lilianweng.github.io', port=443): Max retries exceeded with url: /posts/2023-06-23-agent/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7ffa80323b00>: Failed to resolve 'lilianweng.github.io' ([Errno -3] Temporary failure in name resolution)"))

In [ ]:
text_documents

In [ ]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('GPT4.pdf')
docs=loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:5]

In [ ]:
docs

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

In [ ]:
from langchain_community.vectorstores import Chroma

In [ ]:
db = Chroma.from_documents(documents[:20],OllamaEmbeddings())

In [ ]:
query = "Who are the authors of GPT4 reasearch paper?"

In [ ]:
result = db.similarity_search(query)
result

In [ ]:
from langchain_community.vectorstores import FAISS

db1 = FAISS.from_documents(documents[:20], OllamaEmbeddings())

In [ ]:
db1

In [8]:
from langchain_community.llms import Ollama

In [54]:
llm = Ollama(model = "llama3.2")

/tmp/ipykernel_7436/1028341570.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model = "llama3.2")


In [55]:
from langchain_core.prompts import ChatPromptTemplate

In [57]:
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>
Question: {input}
""")

In [60]:
## Chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [62]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [63]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7f91cba89310>, search_kwargs={})

In [64]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [65]:
response=retrieval_chain.invoke({"input":"Scaled Dot-Product Attention"})

In [66]:
response

{'input': 'Scaled Dot-Product Attention',
 'context': [Document(metadata={'creationdate': '2025-02-16T00:29:33+05:30', 'creator': 'Mozilla Firefox 128.5.0', 'page': 4, 'page_label': '5', 'producer': 'cairo 1.18.0 (https://cairographics.org)', 'source': 'GPT4.pdf', 'total_pages': 100}, page_content='Table 1.GPT performance on academic and professional exams. In each case, we simulate the\nconditions and scoring of the real exam. We report GPT-4’s\ue000nal score graded according to exam-\nspeci\ue000c rubrics, as well as the percentile of test-takers achieving GPT-4’s score.\n5'),
  Document(metadata={'creationdate': '2025-02-16T00:29:33+05:30', 'creator': 'Mozilla Firefox 128.5.0', 'page': 4, 'page_label': '5', 'producer': 'cairo 1.18.0 (https://cairographics.org)', 'source': 'GPT4.pdf', 'total_pages': 100}, page_content='Codeforces Rating 392 (below 5th) 392 (below 5th) 260 (below 5th)\nAP Art History 5 (86th - 100th) 5 (86th - 100th) 5 (86th - 100th)\nAP Biology 5 (85th - 100th) 5 (85

In [67]:
response["answer"]

'Based on the provided context, I will answer the question about Scaled Dot-Product Attention.\n\nHowever, there is no explicit mention of "Scaled Dot-Product Attention" in the provided text. The text mentions that GPT-4 is a Transformer-based model pre-trained to predict the next token in a document, and it does not provide any information about a specific technique called "Scaled Dot-Product Attention".\n\nIf you could provide more context or clarify what Scaled Dot-Product Attention refers to, I\'ll be happy to help.'